# Exploring and loading data from a STAC catalogue

In [16]:
!pip install hdstats

  Using cached hdstats-0.2.1.tar.gz (543 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hdstats: filename=hdstats-0.2.1-cp312-cp312-win_amd64.whl size=951791 sha256=b9344974ec56344063170f18b9374405768ad8ecc4f223024c7864fd0d3b1f4d
  Stored in directory: c:\users\thana\appdata\local\pip\cache\wheels\5b\78\a5\2cea2698e7eb19ef456500bd52ff67cf36daa40e61c359ae4b
Successfully built hdstats


**Keywords** : `data used; sentinel-2`, `data search; STAC`, `platform; Amazon`

## The Idea

The use of the SpatioTemporal Asset Catalog (STAC) to access data from public buckets like AWS, instead of downloading them, offers several key advantages that are critical for efficient data management and usage in geospatial and environmental research. First and foremost, STAC provides a standardized way to search and discover geospatial data stored across different cloud environments. This standardization significantly reduces the complexity and development time required for scientists and developers to interact with vast amounts of Earth Observation data.

By utilizing STAC to access data directly from cloud services like AWS, users can leverage powerful cloud-based tools for processing and analyzing data on-the-fly without the need for extensive local storage or computing resources. This approach not only minimizes data redundancy—by eliminating the need to download large datasets—but also enhances scalability and flexibility in data handling. Researchers can quickly execute queries across large datasets, apply algorithms, and generate insights directly in the cloud, which accelerates the pace of innovation and application in fields such as climate analysis, urban planning, and disaster response.

Moreover, accessing data via STAC from cloud platforms facilitates more dynamic and real-time data analysis. It allows users to integrate the latest data updates automatically and maintain the continuity and accuracy of their analyses. Overall, STAC’s role in connecting users with cloud-hosted public data resources fundamentally shifts the landscape of geospatial data utilization, making it more accessible, cost-effective, and aligned with the needs of a data-driven world.

This notebook showcases:
1. How can we find public collections using pystac library
2. How can we search items inside a catalogue
3. How can we query metadata
4. How can we load directly data into a data cube using open data cube tools

***



### Required Libraries

In [5]:
import pyproj
import pystac_client
from shapely.geometry import box
from shapely.ops import transform
import odc.stac
from odc.geo.geobox import GeoBox

## Exploring Amazon's STAC catalogue 

### Find Collections

In [6]:
STAC_URL = "https://earth-search.aws.element84.com/v0"

client = pystac_client.Client.open(STAC_URL)
collections = client.get_collections()
for collection in collections:
    print(collection)

C:\Users\thana\.conda\envs\odc\Lib\site-packages\pystac_client\client.py:190: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())
C:\Users\thana\.conda\envs\odc\Lib\site-packages\pystac_client\client.py:440: FallbackToPystac: Falling back to pystac. This might be slow.
  self._warn_about_fallback("COLLECTIONS", "FEATURES")


<CollectionClient id=sentinel-s2-l2a>
<CollectionClient id=sentinel-s2-l1c>
<CollectionClient id=sentinel-s2-l2a-cogs>
<CollectionClient id=landsat-8-l1-c1>


### Explore a Catalogue

In [7]:
# v0 collection
STAC_COLLECTION = "sentinel-s2-l2a-cogs"
# v1 collection
# STAC_COLLECTION = "sentinel-2-l2a"
catalog = pystac_client.Client.open(STAC_URL)
catalog.add_conforms_to("ITEM_SEARCH")
catalog.add_conforms_to("QUERY")
catalog

<Client id=earth-search>

### Explore items inside a catalogue

In [8]:
# Get generator of all items in catalog
items = catalog.get_all_items()
item = next(items)
item

<Item id=S2B_15MWT_20230506_0_L2A>

### Contrust a request - query to the catalogue
Items in STAC catalog have much more metadata (in addition to location) that you can query and only return results that match your query parameters. 

First we need to set up some analysis parameters that will be used to search for metadata. 
This includes `product`, which is the same product name used to load data directly using `dc.load`. In this example we'll choose a small area over Athens.

In [9]:
# Parameters that are included in the query

# some spatial projection information
CRS_STRING = "9705"
EPSG = pyproj.CRS.from_string(CRS_STRING).to_epsg()

# Area of Interest
aoi = (23.906250,38.000356,24.004786,38.062149)
aoi_box = box(*aoi)

# Bands
BANDS = ["B04", "B03", "B02"]

# Time range
START_DATE = "2018-07-01"
END_DATE = "2018-08-10"

# STAC items store bounding box info in epsg:4326
STAC_CRS_STRING = "epsg:4326"
transformer_4326 = pyproj.Transformer.from_crs(
    crs_from=EPSG,
    crs_to=STAC_CRS_STRING,
    always_xy=True,
)
bbox_4326 = transform(transformer_4326.transform, aoi_box).bounds
bbox_4326

(23.90625, 38.000356, 24.004786, 38.062149)

In [15]:
!pip install hdstats==0.2.1

  Using cached hdstats-0.2.1.tar.gz (543 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build hdstats


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [39 lines of output]
  {'include_dirs': ['C:\\Users\\thana\\AppData\\Local\\Temp\\pip-build-env-df7kgdql\\overlay\\Lib\\site-packages\\numpy\\_core\\include'], 'extra_compile_args': ['-fopenmp'], 'extra_link_args': ['-fopenmp'], 'define_macros': []}
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\hdstats
  copying hdstats\tsslow.py -> build\lib.win-amd64-cpython-312\hdstats
  copying hdstats\utils.py -> build\lib.win-amd64-cpython-312\hdstats
  copying hdstats\__init__.py -> build\lib.win-amd64-cpython-312\hdstats
  running egg_info
  writing hdstats.egg-info\PKG-INFO
  writing dependency_links to hdstats.egg-info\dependency_links.txt
  writing requirements to hdstats.egg-info\requires.txt
  writing top-level names to hdstats.egg-info\top_level.txt
  reading manifest file 'hdstats.egg-info\SOURCES.txt'
  reading man

In [10]:
stac_items = catalog.search(
    collections=[STAC_COLLECTION],
    bbox=bbox_4326,
    datetime=[START_DATE, END_DATE],
    query={"eo:cloud_cover": {"lt": 10}}
)

# Deprecated:
# feature_collection = stac_items.get_all_items_as_dict()

feature_collection = stac_items.item_collection_as_dict()
feature_collection

{'type': 'FeatureCollection',
 'features': [{'stac_version': '1.0.0-beta.2',
   'assets': {'overview': {'proj:shape': [343, 343],
     'proj:transform': [320, 0, 199980, 0, -320, 4300020, 0, 0, 1],
     'roles': ['overview'],
     'eo:bands': [{'full_width_half_max': 0.038,
       'center_wavelength': 0.6645,
       'name': 'B04',
       'common_name': 'red'},
      {'full_width_half_max': 0.045,
       'center_wavelength': 0.56,
       'name': 'B03',
       'common_name': 'green'},
      {'full_width_half_max': 0.098,
       'center_wavelength': 0.4966,
       'name': 'B02',
       'common_name': 'blue'}],
     'gsd': 10,
     'href': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/KC/2018/8/S2A_35SKC_20180809_0_L2A/L2A_PVI.tif',
     'title': 'True color image',
     'type': 'image/tiff; application=geotiff; profile=cloud-optimized'},
    'thumbnail': {'roles': ['thumbnail'],
     'href': 'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/35/S/KC/2018/8/

In [11]:
# Deprecated old version:
# assets = stac_items.get_all_items()[0].assets

assets = stac_items.item_collection()[0].assets

for key, asset in assets.items():
    print(f"{key}: {asset.title}")

overview: True color image
thumbnail: Thumbnail
metadata: Original XML metadata
B11: Band 11 (swir16)
B01: Band 1 (coastal)
B12: Band 12 (swir22)
B02: Band 2 (blue)
B03: Band 3 (green)
B04: Band 4 (red)
AOT: Aerosol Optical Thickness (AOT)
B05: Band 5
B06: Band 6
B07: Band 7
B08: Band 8 (nir)
B8A: Band 8A
B09: Band 9
WVP: Water Vapour (WVP)
visual: True color image
SCL: Scene Classification Map (SCL)
info: Original JSON metadata


## Load Data directly in an xarray using ODC stac component

The code snippet below showcases how the ODC software, with its STAC component, can be adeptly utilized to manage and analyze satellite imagery data at varying resolutions. The snippet begins by defining `spatial resolutions`—approximately 10 meters—highlighting ODC's capability to handle data granularity based on specific research or operational requirements. The use of a `projection system` (EPSG:4326) for the coordinate reference system is typical for global datasets and ensures that data from various sources aligns correctly on a standard geographic grid.

The instantiation of a `GeoBox` object with these parameters effectively sets up a spatial query box, which filters the data to the defined bounding box, thus optimizing data loading processes by eliminating unnecessary data outside of the interest area. This approach is particularly beneficial in cloud computing environments, where data transfer and processing times are critical cost factors.

The `odc.stac.load` function call is pivotal, demonstrating how STAC items are loaded with specified assets (red, green, blue) and other parameters. The use of chunking enhances performance for large datasets, and specifying a geobox ensures that only the relevant spatial extent is loaded. Notably, the function filters for `specific bands`, and employs `bilinear resampling` to interpolate pixel values, enhancing the visual quality of the imagery.

Lastly, grouping by `solar_day` is a clever use of the data's temporal dimension to mitigate the issue of duplicate observations due to satellite overlap. This functionality ensures that each dataset represents a unique snapshot in time, which is crucial for time-series analysis and change detection studies. Overall, this code is a succinct demonstration of how modern geospatial data workflows can be streamlined through the integration of ODC and STAC, facilitating efficient, scalable, and precise environmental monitoring and analysis.

In [12]:
# using Open Data Cube stac component
dx = 3/3600  # ~90m resolution
# dx = 10 / 111320 # ~10m res
epsg = 4326
geobox = GeoBox.from_bbox(aoi, crs=f"epsg:{epsg}", resolution=dx)

data = odc.stac.load(
    stac_items.items(),
    assets=["red", "green", "blue"],
    chunks={},
    geobox=geobox,
    bands=BANDS,
    resampling="bilinear",
    groupby="solar_day" # delete duplicates due to satellite overlap
)

In [13]:
data

<xarray.Dataset> Size: 644kB
Dimensions:      (latitude: 75, longitude: 119, time: 6)
Coordinates:
  * latitude     (latitude) float64 600B 38.06 38.06 38.06 ... 38.0 38.0 38.0
  * longitude    (longitude) float64 952B 23.91 23.91 23.91 ... 24.0 24.0 24.0
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 48B 2018-07-05T09:17:02 ... 2018-08-09...
Data variables:
    B04          (time, latitude, longitude) float32 214kB dask.array<chunksize=(1, 75, 119), meta=np.ndarray>
    B03          (time, latitude, longitude) float32 214kB dask.array<chunksize=(1, 75, 119), meta=np.ndarray>
    B02          (time, latitude, longitude) float32 214kB dask.array<chunksize=(1, 75, 119), meta=np.ndarray>